In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import beta       
import seaborn as sns
import random
from sklearn.model_selection import train_test_split

: 

Getting dataset from the files

In [ ]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='latin1')
        # X=datadict['data'].reshape((len(datadict['data']), 3, 32, 32)).transpose(0, 2, 3, 1)
        # dict_new=dict['data'].reshape((len(dict['data']), 3, 32, 32)).transpose(0, 2, 3, 1)
    return dict

def NormalizeData(X):
    X = X.astype('float64')
    X /= np.std(X, axis = 0)
batch_1 = unpickle("data_batch_1")
batch_2 = unpickle("data_batch_2")
batch_3 = unpickle("data_batch_3")
batch_4 = unpickle("data_batch_4")
batch_5 = unpickle("data_batch_5")
# batch_test = unpickle("test_batch")
# batch_meta = unpickle("batches.meta")
features_1 = np.array(batch_1['data'].reshape((len(batch_1['data']), 3, 32, 32)).transpose(0, 2, 3, 1))
features_2 = np.array(batch_2['data'].reshape((len(batch_2['data']), 3, 32, 32)).transpose(0, 2, 3, 1))
features_3 = np.array(batch_3['data'].reshape((len(batch_3['data']), 3, 32, 32)).transpose(0, 2, 3, 1))
features_4 = np.array(batch_4['data'].reshape((len(batch_4['data']), 3, 32, 32)).transpose(0, 2, 3, 1))
features_5 = np.array(batch_5['data'].reshape((len(batch_5['data']), 3, 32, 32)).transpose(0, 2, 3, 1))
NormalizeData(features_1[:])
NormalizeData(features_2[:])
NormalizeData(features_3[:])
NormalizeData(features_4[:])
NormalizeData(features_5[:])



: 

In [ ]:
class MISC:
    def loss_Func(self, yh, y):

        return None
       
    def cost_derivative(self, aFunc, y):
        derivative = aFunc - y
        return derivative
    
    def sigmoid(self, z):
        return 1.0/(1.0+np.exp(-z))
    def sigmoid_derivative(self, z):
        return self.sigmoid(z)*(1-self.sigmoid(z))

    def relu(z):
      return np.maximum(0,z)
    def relu_derivative(z):
        z[z<=0] = 0
        z[z>0] = 1
        return z

    def tanh(z):
	    return (np.exp(z) - np.exp(-z)) / (np.exp(z) + np.exp(-z))
    def tanh_derivative(self, z):
        return 1 - np.power(self.tanh(z), 2)

    def softmax(x):
      e = np.exp(x-np.max(x))
      s = np.sum(e, axis=1, keepdims=True)
      return e/s   
    def softmax_derivative(self, z):
       return self.softmax(z)*self.softmax(1-z)



: 

Multilayer perceptron algorithm:

In [ ]:
class MLP:
    
    def __init__(self, aFunc, numOfHiddenLayers, numOfHiddenUnits):
        self.aFunc = aFunc
        self.numOfHiddenLayers = numOfHiddenLayers
        self.numOfHiddenUnits = numOfHiddenUnits

        self.weights = {}
        self.biases = {}

        # Initialize biases for input layer
        self.biases.append(np.zeros(self.numOfHiddenUnits[0]))
        
        # Initialize weights and biases for hidden layers
        for i in range(1, self.numOfHiddenLayers):
            self.weights.append(np.random.randn(self.numOfHiddenUnits[i], self.numOfHiddenUnits[i-1]))
            self.biases.append(np.zeros(self.numOfHiddenUnits[i]))
        
        # Initialize weights and biases for output layer
        self.weights.append(np.random.randn(10, self.numOfHiddenUnits[-1]))
        self.biases.append(np.zeros(10))
            
    def fit(self, x, y, lr, epoch, optimizer):
        
        def backprop(self, x, y):
            z = {}
            a = {1: x}  

            for i in range(1, self.numOfHiddenLayers):
                z[i + 1] = np.dot(a[i], self.weights[i]) + self.biases[i]
                a[i + 1] = self.aFunc[i + 1](z[i + 1])

            delta = self.cost_derivative(a[-1], y) * MISC.softmax_derivative(z[-1])
            dw = np.dot(a[self.numOfHiddenLayers - 1].T, delta)
        
            update_params = {
                self.numOfHiddenLayers - 1: (dw, delta)
            }

            for i in reversed(range(2, self.numOfHiddenLayers)):
                dr = None
                if (self.aFunc[i].equals("Relu")): dr = MISC.relu_derivative(z[i])
                elif (self.aFunc[i].equals("Sigmoid")): dr = MISC.sigmoid_derivative(z[i])
                else: dr = MISC.tanh_derivative(z[i])
                delta = np.dot(delta, self.weights[i].T) * dr
                dw = np.dot(a[i - 1].T, delta)
                update_params[i - 1] = (dw, delta)
	
            # for k, v in update_params.items():
            #     self._update_w_b(k, v[0], v[1])


        def gradient(x, y, params):
           
            return None
              
        params0 = []
        self.params = optimizer.run(gradient, x, y, params0)
        return self
            
    def predict(self, x):
        z = x
        for v in self.params[:-1]:
            z = self.aFunc(np.dot(x, v)) #N x M
            x = z
        yh = softmax(np.dot(z, self.params[-1]))#N
        return yh    

: 

GradientDescent Algorithm

In [ ]:
class GradientDescent:
    
    def __init__(self, learning_rate=.001, batch_size=16, max_iters=1e4, epsilon=1e-8):
        self.learning_rate = learning_rate
        self.max_iters = max_iters
        self.epsilon = epsilon
        self.batch_size = batch_size
        
    def run(self, gradient_fn, x, y, params):
        m = x.shape
        num_batches = m[0] // self.batch_size
        norms = np.array([np.inf])
        t = 1
        while np.any(norms > self.epsilon) and t < self.max_iters:
            permutation = np.random.permutation(m[1])
            x_shuffle = x[:, permutation]
            y_shuffle = y[:, permutation]
            
            for a in range(num_batches):
                start = a * self.batch_size
                end = start + self.batch_size
                x_batch = x_shuffle[start:end,:]
                y_batch = y_shuffle[start:end,:]

                # Compute the gradient 
                grad = gradient_fn(x_batch, y_batch, params)
                for p in range(len(params)):
                    params[p] -= self.learning_rate * grad[p]
                norms = np.array([np.linalg.norm(g) for g in grad])
            t += 1
        return params